In [1]:
#Preparamos la importacion de librerias e instalamos las que nos falten
import sys
import subprocess
import pkg_resources

required  = {'numpy', 'pandas', 'datetime'} 
installed = {pkg.key for pkg in pkg_resources.working_set}
missing   = required - installed

if missing:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', *missing])

In [2]:
#Importamos las librerias luego de la instalacion
import os
import pandas as pd
import numpy as np
import datetime

In [3]:
#Creamos los dataframes eliminando los duplicados
df_netflix_titles = pd.read_json(r'/home/guille/Proyectos/pi01_data05_O/Datasets/netflix_titles.json').drop_duplicates()
df_amazon_prime_titles = pd.read_csv(r'/home/guille/Proyectos/pi01_data05_O/Datasets/amazon_prime_titles.csv').drop_duplicates()
df_disney_plus_titles = pd.read_csv(r'/home/guille/Proyectos/pi01_data05_O/Datasets/disney_plus_titles.csv').drop_duplicates()
df_hulu_titles = pd.read_csv(r'/home/guille/Proyectos/pi01_data05_O/Datasets/hulu_titles.csv').drop_duplicates()

In [4]:
# Se crea columna plataforma
df_netflix_titles['plataforma'] = 'netflix'
df_amazon_prime_titles['plataforma'] = 'amazon'
df_disney_plus_titles['plataforma'] = 'disney'
df_hulu_titles['plataforma'] = 'hulu'

In [5]:
# Se concatena cada dateframe verticalmente, ya que coinciden los nombres de las columnas
df_plataformas_concatenadas= pd.concat([df_netflix_titles,df_amazon_prime_titles,df_disney_plus_titles,df_hulu_titles])


Variables objetivos a tener en cuenta : plataforma - release_year  - duration - time_unit - cast.

In [6]:
# Se normaliza el dataframe concatenado
df_plataformas_concatenadas['date_added'] = pd.to_datetime(df_plataformas_concatenadas["date_added"])
df_plataformas_concatenadas['release_year'] = pd.to_numeric(df_plataformas_concatenadas['release_year'],downcast="integer")
# Se esplitea la columna 'duration' para obtener una columna cuantitativa y otra categorica
cat = df_plataformas_concatenadas['duration'].str.split(" ", n = 1, expand =True) 
df_plataformas_concatenadas['duration']= cat[0] 
df_plataformas_concatenadas['time_unit']= cat[1] 
# Se reemplazan los valores nulos por 0
df_plataformas_concatenadas['duration'].fillna(value=0,inplace=True)
# Se convierte a entero
df_plataformas_concatenadas = df_plataformas_concatenadas.astype({"duration": int})


In [7]:
# Se guarda el dataframe en un csv que va utilizar fastapi para realizar las consultas y como resguardo
df_plataformas_concatenadas.to_csv('df_plataformas_concatenadas', encoding='utf-8')

Declaración de las funciones requeridas y sus respectivos testeos.

In [8]:
# Funcion para obtener titulo de máxima duración según tipo de film (película/serie), por plataforma y por año
def get_max_duration(año,plataforma,unit_time):
    df_plataforma = df_plataformas_concatenadas.query("plataforma == @plataforma and release_year == @año and time_unit == @unit_time")
    maximo = df_plataforma['duration'].idxmax()
    fila = df_plataforma.loc[maximo]
    return(fila['title'])


In [9]:
print(get_max_duration(2018,'hulu','min'))

The House That Jack Built


In [10]:
# Funcion para obtener cantidad de películas y series (separado) por plataforma 
def get_count_plataform (plataforma):
    df_plataforma = df_plataformas_concatenadas.query("plataforma == @plataforma")
    contador = df_plataforma.type.value_counts(sort=True)
    return (plataforma,contador[0],contador[1])
  

In [11]:
print(get_count_plataform('netflix'))

('netflix', 6131, 2676)


In [12]:
# Funcion para obtener Cantidad de veces que se repite un género y plataforma con mayor frecuencia del mismo
def get_listedin (plataforma,categoria):
    df_plataforma = df_plataformas_concatenadas.query("plataforma == @plataforma")
    return (plataforma, len(df_plataforma[df_plataforma.listed_in.str.contains(categoria)]))

In [13]:
print(get_listedin('amazon','Comedy'))

('amazon', 2099)


In [14]:
# Funcion para obtener Actor que más se repite según plataforma y año
def get_actor(plataforma,año):
    df_plataforma = df_plataformas_concatenadas.query("plataforma == @plataforma and release_year == @año")
    new_df = pd.DataFrame(df_plataforma["cast"].str.split(',', expand=True).stack(), columns=["C1"])
    contador = new_df['C1'].value_counts()
    return (contador.index[0],contador[0])


In [15]:
print(get_actor('netflix',2018))

(' Andrea Libman', 8)
